# **Install/Import required Python packages**
You'll need Python (3.7+) and the following packages:
* `pandas` (data structures)
* `lightgbm` (tree-based learning algorithms)
* `scikit-learn`
* `tqdm` (progress bar)

For this project, we will be using game log data from KaggleHub, specifically the [2023-24](https://www.kaggle.com/datasets/joopaivaaaaaaa/nba-gamelogs-2023-24) and [2024-25](https://www.kaggle.com/datasets/joopaivaaaaaaa/nba-gamelogs-2024-25) NBA seasons, to build a dataset for fantasy basketball analysis.

In [1]:
pip install pandas lightgbm scikit-learn tqdm

In [2]:
import pandas as pd
import numpy as np
import kagglehub
import unicodedata
from tqdm import tqdm
import os
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from concurrent.futures import ThreadPoolExecutor, as_completed

# **Fantasy Basketball Scoring**

We will use the default ESPN Fantasy Basketball scoring system as our target for prediction ([source](https://www.espn.com/fantasy/basketball/story/_/id/30296896/espn-fantasy-default-points-league-scoring-explained)):

* PTS = +1 (points scored)
* FG3M = +1 (3 pointers made)
* FGA = -1 (field goals attempted)
* FGM = +2 (field goals made)
* FTA = -1 (free throws attempted)
* FTM = +1 (free throws made)
* REB = +1 (rebounds)
* AST = +2 (assists)
* STL = +4 (steals)
* BLK = +4 (blocks)
* TOV = -2 (turnovers)

Additionally, we will gather player game logs from the last 3 NBA seasons to train the model and predict fantasy performance for the upcoming 2025–26 NBA season.

In [3]:
FANTASY_SCORING = {
    "PTS": 1,
    "FG3M": 1,
    "FGA": -1,
    "FGM": 2,
    "FTA": -1,
    "FTM": 1,
    "REB": 1,
    "AST": 2,
    "STL": 4,
    "BLK": 4,
    "TOV": -2
}

# **Helper Functions**

These functions are used to prepare and process the raw NBA data before training the model:

`compute_fantasy_points` – Calculates player's fantasy points for given game using the default ESPN Fantasy Basketball scoring system. These fantasy points serve as the target variable for the model.

In [4]:
def compute_fantasy_points(df):
    """
    Compute fantasy points for a player.
    """
    # Compute fantasy points
    fantasy_score = (
        df["points"] * FANTASY_SCORING["PTS"] +
        df["threePointersMade"] * FANTASY_SCORING["FG3M"] +
        df["fieldGoalsAttempted"] * FANTASY_SCORING["FGA"] +
        df["fieldGoalsMade"] * FANTASY_SCORING["FGM"] +
        df["freeThrowsAttempted"] * FANTASY_SCORING["FTA"] +
        df["freeThrowsMade"] * FANTASY_SCORING["FTM"] +
        df["reboundsTotal"] * FANTASY_SCORING["REB"] +
        df["assists"] * FANTASY_SCORING["AST"] +
        df["steals"] * FANTASY_SCORING["STL"] +
        df["blocks"] * FANTASY_SCORING["BLK"] +
        df["turnovers"] * FANTASY_SCORING["TOV"]
    )

    df["fantasyPoints"] = fantasy_score

    return df

# **Build dataset for all active NBA players**

Let's prepare the dataset for all active NBA players that will be used to train our Gradient Boosting Decision Tree (GBDT) model.

As part of the data preparation, we will be using prepared Kaggle data that includes player game logs from NBA.com using `nba_api` and extract only the relevant features for modeling. Additionally, we will assign separate weights to each of the past 3 seasons, giving the more recent 2024-25 season greater influence on the model while still incorporating the previous seasons.

Start and end dates for the NBA seasons are taken from [Basketball Reference](https://www.basketball-reference.com/).

In [5]:
# Download the datasets
path = kagglehub.dataset_download("eoinamoore/historical-nba-data-and-player-box-scores")
path_players = kagglehub.dataset_download("yagizfiratt/nba-players-database")

csv = os.path.join(path, "PlayerStatistics.csv")
players_csv = os.path.join(path_players, "PlayerIndex_nba_stats.csv")

df = pd.read_csv(csv)
df["gameDate"] = pd.to_datetime(df["gameDate"])

df_players = pd.read_csv(players_csv)
df_players.rename(columns={"PERSON_ID": "personId", "POSITION": "position"}, inplace=True)

print(df_players.head())

# Define start and end dates for 2022-23 regular season
start_date_22_23 = pd.Timestamp("2022-10-18 00:00:00")
end_date_22_23 = pd.Timestamp("2023-04-09 23:59:59")

# Define start and end dates for 2023-24 regular season
start_date_23_24 = pd.Timestamp("2023-10-24 00:00:00")
end_date_23_24 = pd.Timestamp("2024-04-14 23:59:59")

# Define start and end dates for 2024-25 regular season
start_date_24_25 = pd.Timestamp("2024-10-22 00:00:00")
end_date_24_25 = pd.Timestamp("2025-04-13 23:59:59")

df_22_23 = df[(df["gameDate"] >= start_date_22_23) & (df["gameDate"] <= end_date_22_23)]
df_23_24 = df[(df["gameDate"] >= start_date_23_24) & (df["gameDate"] <= end_date_23_24)]
df_24_25 = df[(df["gameDate"] >= start_date_24_25) & (df["gameDate"] <= end_date_24_25)]

# Merge new position column with each seasons' dataframe based on personId
dataframes = [df_22_23, df_23_24, df_24_25]

for i, df_season in enumerate(dataframes):
    dataframes[i] = df_season.merge(
        df_players[["personId", "position"]],
        on="personId",
        how="left"
    )

# Unpack back into individual variables if needed
df_22_23, df_23_24, df_24_25 = dataframes

# Combine firstName and lastName into playerName
for df_season in [df_22_23, df_23_24, df_24_25]:
    df_season["playerName"] = df_season["firstName"] + " " + df_season["lastName"]

# Preview the data
print("2022-23 season sample:")
print(df_22_23.head())

print("\n2023-24 season sample:")
print(df_23_24.head())

print("\n2024-25 season sample:")
print(df_24_25.head())

100%|██████████| 93.0M/93.0M [00:00<00:00, 115MB/s]

Extracting files...


100%|██████████| 200k/200k [00:00<00:00, 45.8MB/s]

Extracting files...



/tmp/ipython-input-227379080.py:8: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv)


   personId PLAYER_LAST_NAME PLAYER_FIRST_NAME          PLAYER_SLUG  \
0     76001        Abdelnaby              Alaa       alaa-abdelnaby   
1     76002       Abdul-Aziz              Zaid      zaid-abdul-aziz   
2     76003     Abdul-Jabbar            Kareem  kareem-abdul-jabbar   
3        51       Abdul-Rauf           Mahmoud   mahmoud-abdul-rauf   
4      1505      Abdul-Wahad             Tariq    tariq-abdul-wahad   

      TEAM_ID TEAM_SLUG  IS_DEFUNCT    TEAM_CITY      TEAM_NAME  \
0  1610612757   blazers           0     Portland  Trail Blazers   
1  1610612745   rockets           0      Houston        Rockets   
2  1610612747    lakers           0  Los Angeles         Lakers   
3  1610612743   nuggets           0       Denver        Nuggets   
4  1610612758     kings           0   Sacramento          Kings   

  TEAM_ABBREVIATION  ... DRAFT_YEAR DRAFT_ROUND DRAFT_NUMBER  ROSTER_STATUS  \
0               POR  ...     1990.0         1.0         25.0            NaN   
1           

In [6]:
# Define relevant columns to keep
cols_to_keep = [
    "playerName", "playerteamName", "position",
    "points", "threePointersMade", "fieldGoalsAttempted",
    "fieldGoalsMade", "freeThrowsAttempted", "freeThrowsMade",
    "reboundsTotal", "assists", "steals", "blocks", "turnovers"
]

# Ensure numeric columns
numeric_cols = ["points", "threePointersMade", "fieldGoalsAttempted",
    "fieldGoalsMade", "freeThrowsAttempted", "freeThrowsMade",
    "reboundsTotal", "assists", "steals", "blocks", "turnovers"]

df_22_23[numeric_cols] = df_22_23[numeric_cols].apply(pd.to_numeric, errors="coerce")
df_23_24[numeric_cols] = df_23_24[numeric_cols].apply(pd.to_numeric, errors="coerce")
df_24_25[numeric_cols] = df_24_25[numeric_cols].apply(pd.to_numeric, errors="coerce")

# Load CSVs into Pandas with only columns we want to keep
df_22_23 = df_22_23[cols_to_keep]
df_22_23["seasonYear"] = "2022-23"

df_23_24 = df_23_24[cols_to_keep]
df_23_24["seasonYear"] = "2023-24"

df_24_25 = df_24_25[cols_to_keep]
df_24_25["seasonYear"] = "2024-25"

# Compute total and average Fantasy Points for each season separately
df_22_23 = compute_fantasy_points(df_22_23)
df_23_24 = compute_fantasy_points(df_23_24)
df_24_25 = compute_fantasy_points(df_24_25)

# Apply weights for each season
df_22_23["seasonWeight"] = 0.2
df_23_24["seasonWeight"] = 0.3
df_24_25["seasonWeight"] = 0.6

# Combine the datasets
df = pd.concat([df_22_23, df_23_24, df_24_25], ignore_index=True)
df = df.sort_values(by="fantasyPoints", ascending=False).reset_index(drop=True)

print(df.head())
print("Dataset shape:", df.shape)

      playerName playerteamName position  points  threePointersMade  \
0    Luka Doncic      Mavericks      F-G    50.0                8.0   
1    Luka Doncic      Mavericks      F-G    73.0                8.0   
2    Joel Embiid          76ers      C-F    59.0                1.0   
3    Luka Doncic      Mavericks      F-G    60.0                2.0   
4  Stephen Curry       Warriors        G    52.0               12.0   

   fieldGoalsAttempted  fieldGoalsMade  freeThrowsAttempted  freeThrowsMade  \
0                 25.0            15.0                 12.0            12.0   
1                 33.0            25.0                 16.0            15.0   
2                 28.0            19.0                 24.0            20.0   
3                 31.0            21.0                 22.0            16.0   
4                 31.0            16.0                  8.0             8.0   

   reboundsTotal  assists  steals  blocks  turnovers seasonYear  \
0            6.0     15.0     4

/tmp/ipython-input-868109903.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_22_23["seasonYear"] = "2022-23"
/tmp/ipython-input-868109903.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_23_24["seasonYear"] = "2023-24"
/tmp/ipython-input-868109903.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

# **Train Gradient Boosting Model**

A LightGBM regression model is trained to predict fantasy points per game using the dataset of NBA player stats we just built. LightGBM is a type of Gradient Boosting Decision Tree (GBDT) that works particularly well for structured tabular data like box scores.

`num_boost_round=500` sets the maximum number of boosting iterations (trees) to learn complex patterns in the data.

`early_stopping_rounds=50` ensures training stops if the validation error does not improve for 50 consecutive iterations, helping prevent overfitting.

`verbose_eval=100` prints progress every 100 iterations so we can monitor the model’s performance.

After training, we evaluate the model using Root Mean Squared Error (RMSE), which measures how far the predicted fantasy points deviate from the actual values. A lower RMSE indicates better predictive accuracy.

In [7]:
X = df[numeric_cols]
y = df["fantasyPoints"]
weights = df["seasonWeight"]

X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(
    X, y, weights, test_size=0.2, random_state=42, shuffle=True
)

# lgb refers to LightGBM, a popular gradient boosting framework from Microsoft
train_data = lgb.Dataset(X_train, label=y_train, weight=w_train)
test_data = lgb.Dataset(X_test, label=y_test, weight=w_test, reference=train_data)

params = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.05,
    "num_leaves": 31,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "min_data_in_leaf": 5,
    "verbose": -1
}

# Use a callback for early stopping
callbacks = [
    lgb.early_stopping(stopping_rounds=50),
    lgb.log_evaluation(period=100)
]

# Train the model
model = lgb.train(
    params,
    train_data,
    valid_sets=[test_data],
    num_boost_round=500,
    callbacks=callbacks
)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# Evaluate accuracy of model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)

Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 1.99009
[200]	valid_0's rmse: 1.04564
[300]	valid_0's rmse: 0.790337
[400]	valid_0's rmse: 0.687911
[500]	valid_0's rmse: 0.63825
Did not meet early stopping. Best iteration is:
[500]	valid_0's rmse: 0.63825
RMSE: 0.6382481395456827


# **Get Player Rankings**

This step produces the final rankings that can be used for fantasy basketball drafts or lineup decisions, taking into account player performance trends, positional roles, and season weights.

In [9]:
df["predictedFantasyPoints"] = model.predict(df[numeric_cols])
player_ranks = df.groupby(["playerName"]).agg(
    predictedFantasyPoints=("predictedFantasyPoints", "mean"),
    fantasyPoints=("fantasyPoints", "mean"),
    position=("position", "last")
).reset_index()

players = player_ranks.sort_values(by="predictedFantasyPoints", ascending=False).reset_index(drop=True)

# Separate by position
guards = player_ranks[player_ranks["position"] == "G"].sort_values(by="predictedFantasyPoints", ascending=False).reset_index(drop=True)
guards_forwards = player_ranks[player_ranks["position"].isin(["G-F", "F-G"])].sort_values(by="predictedFantasyPoints", ascending=False).reset_index(drop=True)
forwards = player_ranks[player_ranks["position"] == "F"].sort_values(by="predictedFantasyPoints", ascending=False).reset_index(drop=True)
fowards_centers = player_ranks[player_ranks["position"].isin(["F-C", "C-F"])].sort_values(by="predictedFantasyPoints", ascending=False).reset_index(drop=True)
centers = player_ranks[player_ranks["position"] == "C"].sort_values(by="predictedFantasyPoints", ascending=False).reset_index(drop=True)

In [10]:
print("Top Players:")
print(players.head(25)[["playerName", "predictedFantasyPoints"]])

print("\nTop Guards:")
print(guards.head(10)[["playerName", "predictedFantasyPoints"]])

print("\nTop Guards/Forwards:")
print(guards_forwards.head(10)[["playerName", "predictedFantasyPoints"]])

print("\nTop Forwards:")
print(forwards.head(10)[["playerName", "predictedFantasyPoints"]])

print("\nTop Forwards/Centers:")
print(fowards_centers.head(10)[["playerName", "predictedFantasyPoints"]])

print("\nTop Centers:")
print(centers.head(10)[["playerName", "predictedFantasyPoints"]])

Top Players:
                 playerName  predictedFantasyPoints
0              Nikola Jokic               63.661640
1               Luka Doncic               58.718196
2     Giannis Antetokounmpo               55.705570
3   Shai Gilgeous-Alexander               55.065750
4         Victor Wembanyama               52.752966
5             Anthony Davis               51.472981
6               Joel Embiid               49.976682
7              LeBron James               49.628619
8         Tyrese Haliburton               48.202626
9          Domantas Sabonis               47.710981
10             Kevin Durant               47.506675
11             Jayson Tatum               46.911332
12             Kyrie Irving               45.477269
13            Stephen Curry               44.658458
14             De'Aaron Fox               44.121737
15         Donovan Mitchell               43.961669
16             James Harden               43.862357
17              LaMelo Ball               43.737964